In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/testdata/testData.tsv
/kaggle/input/unlabeledtraindata/unlabeledTrainData.tsv
/kaggle/input/labeledtraindata/labeledTrainData.tsv


In [2]:
import pandas as pd
train=pd.read_csv("/kaggle/input/labeledtraindata/labeledTrainData.tsv",header=0,delimiter="\t",quoting=3)
test = pd.read_csv("/kaggle/input/testdata/testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("/kaggle/input/unlabeledtraindata/unlabeledTrainData.tsv", header=0,delimiter="\t", quoting=3)

In [3]:
print(train["review"].size)
print(test["review"].size)
print(unlabeled_train["review"].size)

25000
25000
50000


In [4]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [5]:
def review_to_wordlist(review, remove_stopwords=False):
    # 函数将文档转换为单词序列，可选择性地移除停用词。返回单词列表。
    #
    # 1. 移除HTML
    review_text = BeautifulSoup(review).get_text()
    #
    # 2. 移除非字母字符
    review_text = re.sub("[^a-zA-Z]", " ", review_text)
    #
    # 3. 将单词转换为小写并分割它们
    words = review_text.lower().split()
    #
    # 4. 可选择性地移除停用词（默认为False）
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. 返回单词列表
    return words

In [6]:
import nltk.data

# 加载punkt分词器
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [7]:
def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences=[]
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
             sentences.append(review_to_wordlist(raw_sentence,remove_stopwords))
    return sentences

In [8]:
#特征工程 处理数据
sentences=[]
for review in train["review"]:
    sentences+=review_to_sentences(review,tokenizer)
#print(sentences)

/tmp/ipykernel_18/3932766883.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()
/tmp/ipykernel_18/3932766883.py:5: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()


In [9]:
for review in unlabeled_train["review"]:
    sentences+=review_to_sentences(review,tokenizer)

/tmp/ipykernel_18/3932766883.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()
/tmp/ipykernel_18/3932766883.py:5: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()


In [10]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)
# 设置各种参数的值
num_features = 300    # 词向量维度
min_word_count = 40   # 最小词数
num_workers = 4       # 并行运行的线程数
context = 10          # 上下文窗口大小
downsampling = 1e-3   # 频繁词的下采样设置
from gensim.models import word2vec
model = word2vec.Word2Vec(sentences, workers=num_workers, vector_size=num_features, min_count=min_word_count, window=context, sample=downsampling)
model_name = "300features_40minwords_10context"
model.save(model_name)

In [11]:
model.wv.doesnt_match("man woman child kitchen".split())


'kitchen'

In [12]:
type(model.wv.vectors)

numpy.ndarray

In [13]:
model.wv.vectors.shape

(16490, 300)

In [14]:
import numpy as np
def makeFeatureVec(words,model,num_features):#一维向量的平均值
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords=0.
    ubdex2word_list=set(model.index2word)
    for word in words:
        if word in index2word_set: ##如果在词汇表中
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])
    featureVec=np.divide(featureVec,nwords)
    return featureVec

In [15]:
def getAvgFeatureVecs(reviews, model, num_features):
    counter=0.
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype="float32")
    for review in reviews:
        reviewFeatureVecs[counter]=makeFeatureVec(review,model,num_features)
        count=count+1
    return reviewFeatureVecs

In [16]:
clean_train_reviews=[]
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review,remove_stopwords=True))##清理
    trainDataVecs=getAvgFeatureVecs(clean_train_reviews,model,num_features)
clean_test_reviews=[]
for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist(review,remove_stopwords=True))##清理
    testDataVecs=getAvgFeatureVecs(clean_test_reviews,model,num_features)

AttributeError: 'Word2Vec' object has no attribute 'index2word'

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest = RandomForestClassifier(n_estimators=100)
forest = forest.fit(trainDataVecs, train["sentiment"])
result=forest.predict(testDataVecs)
output = pd.DataFrame(data={"id": test["id"], "sentiment": result})
output.to_csv("Word2Vec_AverageVectors.csv", index=False, quoting=3)